# **Coursera Capstone**
### ML Truter

*This notebook documents the final capstone project of the IBM Data Science Professional Certificate*

In [125]:
# Modules for data wrangling
import numpy as np
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # Map plotting

# Module for other functionalty
import os
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Modules for model building
from sklearn.cluster import KMeans

# Settings for modules
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [83]:
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'canadian_cities.csv',
 'capstone_project.ipynb',
 'cost_of_living_index.csv',
 'Geospatial_Coordinates.csv',
 'Neighborhood_Segmentation_Clustering.ipynb',
 'README.md',
 'test.py']

In [98]:
# Load data to be used
df_lcost = pd.read_csv('cost_of_living_index.csv') # Relative living cost metrics relative to New York (baseline = 100)
df_cities = pd.read_csv('canadian_cities.csv') 

In [163]:
# Data Cleaning and merging
# Filter out only canadian cities from the df_lcost df
df_temp = pd.DataFrame()
df_temp = df_lcost.drop('City', axis=1)

df_temp['City'] = df_lcost['City'].apply(lambda x: x.split(',')[0].strip())
df_temp['Country'] = df_lcost['City'].apply(lambda x: x.split(',')[1].strip())

# Only keep Canadian cities 
df_temp = df_temp[df_temp['Country'] == 'Canada']
df_temp['City'] = df_temp['City'].apply(lambda x:x.replace('Quebec City', 'Quebec')) 

# Clean location and population df (naming of Montreal and Quebec)
df_cities['city'] = df_cities['city'].apply(lambda x:x.replace('é', 'e').strip())

# Merge dataframes
df_can = pd.merge(left=df_temp, right=df_cities, left_on='City', right_on='city', how='inner')
df_can = df_can.drop(['city', 'country', 'capital', 'Country', 'iso2', 'population_proper'], axis=1).reset_index()

# Reorder the columns more logically
df_can = df_can[['City', 'admin', 'lat', 'lng', 'population', 'Cost of Living Index', 'Rent Index', 'Local Purchasing Power Index']]
df_can = df_can.rename(columns={'City':'city', 'population':'pop', 'Cost of Living Index':'living_indx', 'Rent Index':'rent_indx', 'Local Purchasing Power Index':'ppi'})

In [164]:
# Top 5 Cities in Canada in terms of PPI
df_can.sort_values('ppi', ascending=False).head(5)

,city,admin,lat,lng,pop,living_indx,rent_indx,ppi
18,Ottawa,Ontario,45.416667,-75.700000,1145000,62.88,30.26,144.70
13,Quebec,Québec,46.800000,-71.250000,624177,65.07,16.10,140.18
7,Red Deer,Alberta,52.266667,-113.800000,74857,67.42,23.58,140.16
5,Calgary,Alberta,51.083333,-114.083333,1110000,68.93,29.85,138.51
23,Kitchener,Ontario,43.446976,-80.472484,417001,58.93,25.38,129.74


In [272]:
# Augment current df with data from the Foursquare API
# Foursquare API user details
CLIENT_ID = 'O0LIFMUBGPNZ4Y5NSWWC23OFU0QM3QMDLX32UN42DOAXXLVB'
CLIENT_CODE = '2J5PWAOXJKSTRSTPZBJMCFP1QMNQUU5NNRGGNSR2R45AK5X3'
VERSION = '20180605'
LIMIT = 100

def search_call_num_locs(city_lat, city_lng, radius, category_ids):
    """ Function that returns the number of venues of interest as defined by the catergory ids within the specified radius."""
   # Search for outdoor trails, ski trails, biking trails, outdoor gyms 
    params = dict(
      client_id=CLIENT_ID,
      client_secret=CLIENT_CODE,
      v=VERSION,
      ll='{}, {}'.format(city_lat, city_lng),
      radius=radius,
      intent='browse',
      categoryId=category_ids,
      limit=LIMIT
    )
    resp = requests.get(url=url_search, params=params)
    data = json.loads(resp.text)
    num_venues = len(data['response']['venues'])
    return num_venues
    

In [285]:
url_explore = 'https://api.foursquare.com/v2/venues/explore'
url_search = 'https://api.foursquare.com/v2/venues/search'

city_locs = pd.DataFrame(columns=['city', 'admin', 'outdoor', 'food', 'work_opp', 'entertainment', 'remoteness'])

# Determine category ids of locations of interest (from: https://developer.foursquare.com/docs/resources/categories)

# List of outdoor locations of interest
outdoor_loc_interest = ['4bf58dd8d48988d159941735', '4eb1c0f63b7b52c0e1adc2eb', '56aa371be4b08b9a8d57355e', '58daa1558bbb0b01f18ec203']
rad_outd = '20000' # Radius to search for outdoor locations (m)
# Food locations of interest
food_loc = ['4d4b7105d754a06374d81259']
rad_food = '3000' 
# Work opportunity locations of interest. Tech startups, corporate cafeteria and corporate coffee shops
work_loc = ['4bf58dd8d48988d125941735', '54f4ba06498e2cf5561da814', '5665c7b9498e7d8a4f2c0f06']
rad_work = '30000' 
# Entertainment locations of interest. Nightlife: Beer bar, whisky bar, nightclubs, wine bar and pubs
entertainm_loc = ['4bf58dd8d48988d122941735', '4bf58dd8d48988d11b941735', '56aa371ce4b08b9a8d57356c', '4bf58dd8d48988d11f941735']
rad_enter = '3000'
# Remoteness locations of interest. Airports
remoteness_loc = ['4bf58dd8d48988d1eb931735']
rad_remoteness = '50000'

for i, city in df_can.iterrows(): # for each city

    num_outdoor_loc = search_call_num_locs(city['lat'], city['lng'], rad_outd, outdoor_loc_interest)
    
    num_food_loc = search_call_num_locs(city['lat'], city['lng'], rad_food, food_loc)

    num_work_loc = search_call_num_locs(city['lat'], city['lng'], rad_work, work_loc)

    num_enter_loc = search_call_num_locs(city['lat'], city['lng'], rad_enter, entertainm_loc)

    num_remoteness_loc = search_call_num_locs(city['lat'], city['lng'], rad_remoteness, remoteness_loc)
    
    # Add data to the dataframe
    city_locs = city_locs.append({'city':city['city'], 'admin':city['admin'], 'outdoor':num_outdoor_loc, 'food':num_food_loc, 'work_opp':num_work_loc, 'entertainment':num_enter_loc, 'remoteness':num_remoteness_loc}, ignore_index=True)
    

In [286]:
city_locs.sort_values('city')

,city,admin,outdoor,food,work_opp,entertainment,remoteness
22,Abbotsford,British Columbia,31,50,0,4,11
0,Barrie,Ontario,15,50,0,9,5
5,Calgary,Alberta,25,50,1,12,42
8,Edmonton,Alberta,45,50,5,27,39
2,Halifax,Nova Scotia,47,30,1,43,16
20,Hamilton,Ontario,50,28,0,27,1
9,Kelowna,British Columbia,30,50,0,11,5
10,Kingston,Ontario,12,10,0,0,2
23,Kitchener,Ontario,50,19,1,6,2
21,London,Ontario,29,17,0,17,3


In [287]:
# Merge city location data with the cost data
df_final = pd.merge(left=df_can, right=city_locs, on=['city', 'admin'], how='inner')
df_final

,city,admin,lat,lng,pop,living_indx,rent_indx,ppi,outdoor,food,work_opp,entertainment,remoteness
0,Barrie,Ontario,44.383333,-79.700000,182041,77.37,29.08,99.44,15,50,0,9,5
1,Saskatoon,Saskatchewan,52.133333,-106.666667,198958,74.26,26.16,103.89,3,50,0,18,11
2,Halifax,Nova Scotia,44.650000,-63.600000,359111,70.88,28.05,99.86,47,30,1,43,16
3,Toronto,Ontario,43.666667,-79.416667,5213000,70.19,44.28,99.45,50,50,4,28,21
4,Vancouver,British Columbia,49.250000,-123.133333,2313328,69.63,49.26,94.65,26,50,3,19,50
5,Calgary,Alberta,51.083333,-114.083333,1110000,68.93,29.85,138.51,25,50,1,12,42
6,Regina,Saskatchewan,50.450000,-104.616667,176183,68.46,28.00,112.05,3,50,0,12,2
7,Red Deer,Alberta,52.266667,-113.800000,74857,67.42,23.58,140.16,6,50,0,9,2
8,Edmonton,Alberta,53.550000,-113.500000,1058000,67.03,29.22,129.45,45,50,5,27,39
9,Kelowna,British Columbia,49.900000,-119.483333,125109,66.60,33.08,108.98,30,50,0,11,5


In [235]:
# Cluster the cities in some usefull way

In [ ]:
X = df_final

In [288]:
df_final.columns

Index(['city', 'admin', 'lat', 'lng', 'pop', 'living_indx', 'rent_indx', 'ppi',
       'outdoor', 'food', 'work_opp', 'entertainment', 'remoteness'],
      dtype='object')